In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def hoocnerChia(a, value):
    b = []
    b.append(a[0])
    for i in range(1, len(a)):
        b.append(b[i - 1] * value + a[i])
    return b

In [ ]:
def hoocnerNhanBosung(coeffPolyTich, t):
    if len(coeffPolyTich) == 0:
        coeffPolyTich.append(1)
        coeffPolyTich.append(-t)
    else:
        coeffPolyTich.append(0)
        temp = coeffPolyTich.copy()
        for i in range(1, len(coeffPolyTich)):
            coeffPolyTich[i] = coeffPolyTich[i] - t * temp[i - 1]
    return coeffPolyTich

In [ ]:
def saiPhan(f_0, f_1, yk, x0, xk):
    if xk > x0:
        temp = f_0.copy()
        f_0.append(0)
        f_0[0] = yk
        for i in range(1, len(f_0)):
            f_0[i] = f_0[i - 1] - temp[i - 1]
        f_1.append(f_0[len(f_0) - 1])
    else:
        temp = f_1.copy()
        f_1.append(0)
        f_1[0] = yk
        for i in range(1, len(f_1)):
            f_1[i] = temp[i - 1] - f_1[i - 1]
        f_0.append(f_1[len(f_1) - 1])
    return f_0, f_1

In [ ]:
def besselFormula(heSoDaThuc, coeffTich, f_0, f_1, k, x0, x1, x2, y1, y2, giaiThua_k):
    #da thuc theo an u
    if k == 0: #Buoc co so
        heSoDaThuc.append(y2 - y1)
        heSoDaThuc.append((y1 + y2)/2)
        f_0.append(y2)
        f_1.append(y1)
        f_0.append(y2 - y1)
        f_1.append(y2 - y1)
        giaiThua_k = 1
    else:
        f_0, f_1 = saiPhan(f_0, f_1, y1, x0, x1)
        f_0, f_1 = saiPhan(f_0, f_1, y2, x0, x2)

        #su dung dat p = u^2: p - 1/4; p - 9/4 ...; p - (k - 1)^2/4
        coeffTich = hoocnerNhanBosung(coeffTich, (k - 1)**2/4)

        giaiThua_k *= len(heSoDaThuc)
        c_chan = (f_1[len(f_1) - 2] + f_0[(len(f_0) - 2)]) / (2 * giaiThua_k)
        giaiThua_k *= len(heSoDaThuc) + 1
        c_le = f_1[len(f_1) - 1] / giaiThua_k
        
        #tinh he so da thuc k le
        heSoDaThucCu = heSoDaThuc.copy()
        heSoDaThuc.append(0)
        for j in range(1, len(heSoDaThuc)):
            if j % 2 == 0:
                heSoDaThuc[j] = heSoDaThucCu[j - 1] + c_chan * coeffTich[int(j / 2)]
            else:
                heSoDaThuc[j] = heSoDaThucCu[j - 1]
        heSoDaThuc[0] = c_chan

        #tinh he so da thuc k chan
        heSoDaThucCu = heSoDaThuc.copy()
        heSoDaThuc.append(0)
        for j in range(1, len(heSoDaThuc)):
            if j % 2 == 0:
                heSoDaThuc[j] = heSoDaThucCu[j - 1] + c_le * coeffTich[int((j + 1) / 2)]
            else:
                heSoDaThuc[j] = heSoDaThucCu[j - 1]
        heSoDaThuc[0] = c_le

    return heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k

In [ ]:
def sortData(path):
    if path.endswith('.csv'):
        data = pd.read_csv(path)
    elif path.endswith('.txt'):
        print("su dung doc file .txt")
    dataX = data['x']
    dataY = data['y']
    for i in range(len(dataX)):
        indexMin = i
        for j in range(i + 1, len(dataX)):
            if dataX[j] < dataX[indexMin]:
                indexMin = j
        if indexMin != i:
            dataX[indexMin], dataX[i] = dataX[i], dataX[indexMin]
            dataY[indexMin], dataY[i] = dataY[i], dataY[indexMin]
    return dataX, dataY

# Nhập mốc nội suy ban đầu

## Truyền dữ liệu vào và hàm kiểm tra điều kiện đầu vào của mốc ban đầu chọn

In [ ]:
path = 'input.csv'
dataX, dataY = sortData(path)
def kiemTraMoc(dataX, x0):
    h = dataX[1] - dataX[0]
    m = round((x0 - dataX[0])/h)
    if x0 > dataX[len(dataX) - 1] or x0 < dataX[0] or x0 - (dataX[0] + m * h) > 1e-10:
        print("Moc noi suy duoc chon ban dau la khong hop le")
        print("Quay lai, chon lai moc noi suy")
    else:
        print("Moc noi suy ban dau hop le. Chuong trinh tiep tuc")

## Chọn mốc nội suy ban đầu

In [ ]:
print("Mang gia tri x", dataX)
print(dataY)

In [ ]:
#Lua chon moc noi suy ban dau
x0 = 3
kiemTraMoc(dataX, x0)

# Chọn số mốc nội suy sử dụng

## Kiểm tra sự hợp lệ của số lượng mốc nội suy lựa chọn

In [ ]:
def checkSoMoc(dataX, soMoc, x0):
    h = dataX[1] - dataX[0]
    m = round((x0 - dataX[0])/h)
    if m <= len(dataX)/2:
        n = 2 * m + 1
    else:
        n = 2 * (len(dataX) - m) + 1
    if n > len(dataX):
        n = len(dataX)
        
    #check xem so moc chon co hop le hay khong
    if soMoc > n or soMoc <= 0 or soMoc % 2 == 1:
        print("so luong moc lua chon la khong hop le")
        print("Nhap lai so moc. So moc phai la so chan thuoc trong khoang tu:", 0, "->", n)
    else:
        print("so luong moc la hop le. Tiep tuc chuong trinh")

## Nhập vào số mốc nội suy lựa chọn

In [ ]:
#lua chon so moc
soMoc = 10
checkSoMoc(dataX, soMoc, x0)

# Sắp xếp lại dữ liệu theo trình tự thêm mốc nội suy

In [ ]:
def sortGauss(dataX, dataY, x0, soMoc):
    x = []
    y = []
    m = round((x0 - dataX[0]) / (dataX[1] - dataX[0]))
    for i in range(soMoc):
        if i % 2 == 1:
            k = int((i + 1)/2)
            x.append(dataX[m + k])
            y.append(dataY[m + k])
        else:
            k = int(-(i + 1)/2)
            x.append(dataX[m + k])
            y.append(dataY[m + k])
    return x, y

# Hàm Main chương trình chính

In [ ]:
def main():
    heSoDaThuc = []
    coeffTich = []
    f_0 = [] 
    f_1 = [] 
    giaiThua_k = 1
    h = dataX[1] - dataX[0]
    with open("output.txt", "w") as out:
        x, y = sortGauss(dataX, dataY, x0, soMoc)
        for i in range(0, soMoc, 2):
            heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k = besselFormula(heSoDaThuc, coeffTich, f_0, f_1, i, x0, x[i], x[i + 1], y[i], y[i + 1], giaiThua_k)
            
            out.writelines("\n\n--------------------------------------------------------------------------------------")
            out.writelines("\nDa thuc moi sau khi them moc 2 noi suy: ({}, {}); ({}, {}): ".format(x[i], y[i], x[i + 1], y[i + 1]))
            out.write("\nTap cac moc noi suy: \t")
            for j in range(i + 2):
                out.write("({}, {}); \t".format(x[j], y[j]))
            out.write("\nDa thuc bac {}: \nP(t) =: ".format(i + 1))
            for j in range(len(heSoDaThuc)):
                out.write("{} * x^{} \t + \t".format(heSoDaThuc[j], len(heSoDaThuc) - 1 - j))
            out.write("\n\nThu lai\n")
            for j in range(i + 2):
                out.writelines("\nTai du lieu x = {}".format(x[j]))
                u = (x[j] - x[0])/h - 1/2
                a = hoocnerChia(heSoDaThuc, u)
                out.writelines("\nPn(x) - y = {}".format(a[len(heSoDaThuc) - 1] - y[j]))
    return heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k

In [ ]:
heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k = main()

# Vẽ biểu đồ kết quả

In [ ]:
x, y = sortGauss(dataX, dataY, x0, soMoc)
h = dataX[1] - dataX[0]
xTest = np.linspace(x[0] - 5*h, x[len(x) - 1] + h, 50)
result = []
for x1 in xTest:
    u = (x1 - x[0])/h - 1/2
    result.append(hoocnerChia(heSoDaThuc, u)[len(heSoDaThuc) - 1])

#ve bieu do duong
plt.plot(xTest, result)
plt.scatter(x, y, color = 'r')
fig = plt.gcf()
fig.set_size_inches(8,5, forward = False)

plt.show()